## OS

In [ ]:
cd ..

## Imports 

In [ ]:
from src.imports import *
from src.transformers import *
from src.helpers import *
from src.vars import *
from src.tools import *

## Data

In [ ]:
train = pd.read_csv("data/source/train.csv")
test = pd.read_csv("data/source/test.csv")

## Split 

In [ ]:
X = train.drop("Transported", axis=1)
y = train["Transported"]

## pipe and params

In [ ]:
# pipeline = Pipeline(
#     [
#         ("column_selector", ColumnSelector()),
#         ("log_transformer", LogTransformer()),
#         ("preprocessor", ColumnTransformer([])),
#         ("sampler_1", RandomUnderSampler()),
#         ("imputer", KNNImputer()),
#         ("scaler", StandardScaler()),
#         ("sampler_2", RandomUnderSampler()),
#         ("estimator", LogisticRegression()),
#     ]
# )

In [ ]:
# param_grid = {
#     "log_transformer__threshold": [0.5, 0.75,1, 1.5],
#     "preprocessor__transformers": [t1],
#     "sampler_1": [, "passthrough"],
#     "scaler": scaler_list,
#     "sampler_2": [RandomUnderSampler(), "passthrough"],
#     "estimator": [
#     LogisticRegression(),
#     RandomForestClassifier(max_depth=128, min_samples_split=4, n_estimators=1280)

#     ],
# }

## Train

In [ ]:
# grid
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv(n=10, test_size=0.2),
    scoring="accuracy",
    n_jobs=12,
    return_train_score=True,
    verbose=1,
    refit=True,
)

# fit
grid.fit(X, y)

# results
res = resultize(grid)

# save & push
save_predict_all(grid)
commit_push(res.mean_test_score.iloc[0])

# display
res

In [ ]:
# grid 2
grid_2 = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,
    scoring="accuracy",
    n_jobs=12,
    return_train_score=True,
    verbose=2,
    refit=True,
)

# fit
grid_2.fit(X, y)

# results
res_2 = resultize(grid_2)

# save & push
save_predict_all(grid_2)
commit_push(res_2.mean_test_score.iloc[0])

# display
res_2

In [ ]:
1 / 0

## fine tuning

In [ ]:
preprocessor = grid.best_estimator_[:-2]
preprocessor

In [ ]:
_X = preprocessor.transform(X)
_X

In [ ]:
pipeline_fine = Pipeline(
    [
        ("sampler", RandomUnderSampler()),
        ("estimator", RandomForestClassifier()),
    ]
)

In [ ]:
param_grid_fine = {
    "estimator__n_estimators": [40, 80, 160, 320, 640, 1280],
    # "estimator__max_depth": [None,  4, 8, 16, 32, 64, 128],
    # "estimator__min_samples_split": [ None, 4,  8, 16, 32, 64, 129],
    # "estimator__oob_score": [True, False],
    # "estimator__criterion": ["gini" ], # "entropy", "log_loss"
}

In [ ]:
cardinality(param_grid_fine)

In [ ]:
grid_fine = HalvingGridSearchCV(
    pipeline_fine,
    param_grid_fine,
    cv=5,
    scoring="accuracy",
    n_jobs=12,
    return_train_score=True,
    verbose=1,
    refit=True,
    aggressive_elimination=True,
)
grid_fine

In [ ]:
grid_fine.fit(_X, y)

In [ ]:
grid_fine.best_estimator_

In [ ]:
res_fine = resultize(grid_fine)
res_fine